# Task 2 RNN

References:
- [ECG Heartbeat Classification: A Deep Transferable Representation](https://arxiv.org/pdf/1805.00794.pdf)

## Load dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

2023-06-08 10:00:36.582292: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## MIT-BIH  Arrhythmia Dataset

- Number of Samples: 109446
- Number of Categories: 5
- Sampling Frequency: 125Hz
- Data Source: Physionet's MIT-BIH Arrhythmia Dataset
- Classes: ['N': 0, 'S': 1, 'V': 2, 'F': 3, 'Q': 4]
- Remark: All the samples are cropped, downsampled and padded with zeroes if necessary to the fixed dimension of 188.
- The final element of each row denotes the class to which that example belongs.


In [4]:
df_mitbih_train = pd.read_csv("../ecg_dataset/mitbih_train.csv", header = None)
df_mitbih_test = pd.read_csv("../ecg_dataset/mitbih_test.csv", header = None)

# print shapes of the dataframes
print("The shape of the mitbih_train is : ", df_mitbih_train.shape)
print("The shape of the mitbih_test is : ", df_mitbih_test.shape)

The shape of the mitbih_train is :  (87554, 188)
The shape of the mitbih_test is :  (21892, 188)


# Build model for classification

Configure training and testing sets. 

In [14]:
X_train, y_train = df_mitbih_train.iloc[: , :-1], df_mitbih_train.iloc[: , -1]

X_test, y_test = df_mitbih_test.iloc[:,:-1], df_mitbih_test.iloc[: , -1]

In [42]:
inputs = tf.keras.Input(shape=(X_train.shape[1],))

expand = tf.expand_dims(inputs, axis=2)
gru = tf.keras.layers.GRU(256, return_sequences=True)(expand)
flatten = tf.keras.layers.Flatten()(gru)

outputs = tf.keras.layers.Dense(5, activation='softmax')(flatten)
outputs1=tf.math.argmax(outputs)
outputs1=tf.cast(outputs1,tf.int32)
model = tf.keras.Model(inputs=inputs, outputs=outputs1)

print(model.summary())

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 187)]             0         
                                                                 
 tf.expand_dims_7 (TFOpLambd  (None, 187, 1)           0         
 a)                                                              
                                                                 
 gru_7 (GRU)                 (None, 187, 256)          198912    
                                                                 
 flatten_7 (Flatten)         (None, 47872)             0         
                                                                 
 dense_7 (Dense)             (None, 5)                 239365    
                                                                 
 tf.math.argmax_2 (TFOpLambd  (5,)                     0         
 a)                                                        

In [45]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
              metrics=['accuracy', 
                       tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
                       )

history = model.fit(
    X_train,
    labels,
    validation_split=0.2,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100


TypeError: in user code:

    File "/Users/bender13/anaconda3/envs/ML/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/Users/bender13/anaconda3/envs/ML/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/bender13/anaconda3/envs/ML/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/Users/bender13/anaconda3/envs/ML/lib/python3.9/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/bender13/anaconda3/envs/ML/lib/python3.9/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/Users/bender13/anaconda3/envs/ML/lib/python3.9/site-packages/keras/eng